In [5]:
from collections import defaultdict
import os, os.path
import re
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from estnltk.resolve_layer_dag import make_resolver
from estnltk.taggers import VabamorfTagger
from estnltk.taggers import VabamorfAnalyzer
from estnltk.taggers import VabamorfCorpusTagger, CorpusBasedMorphDisambiguator
from estnltk import Text

In [6]:
#VabamorfTagger koos tekstipõhise eelühestamise- ja järelühestamisega ning liitsõnade ühestamise heuristikuga. 
cb_disambiguator = CorpusBasedMorphDisambiguator( output_layer='morph_liit', 
                                                  disamb_compound_words=True )
 
eel_jarel_liit = VabamorfCorpusTagger(
    output_layer='morph_liit',
    disambiguate=True,
    guess=True, 
    propername=True,
    slang_lex=False,
    use_postanalysis=True,
    predisambiguate=True,
    postdisambiguate=True,
    cb_disambiguator=cb_disambiguator
    )

In [7]:
# Corpus with gold standard annotations and automaatmärgendus
input_dir = 'Automaatmargendusega_EDT_hastaalfinaal2'
assert os.path.isdir( input_dir )
loaded_texts_edt_am = []
for fname in os.listdir( input_dir ):
    #if 'dev' in fname:
        #continue
    #if 'test' in fname:
     #   continue
    #if 'train' in fname:
     #   continue
    if fname.endswith('.json'):
        # Load Text with gold standard annotations
        text = json_to_text(file=os.path.join(input_dir, fname) )
        loaded_texts_edt_am.append( text )
        print(' Loaded ', fname)

 Loaded  et_edt-ud-dev_0_am.json
 Loaded  et_edt-ud-dev_19_am.json
 Loaded  et_edt-ud-dev_1_am.json
 Loaded  et_edt-ud-dev_20_am.json
 Loaded  et_edt-ud-dev_29_am.json
 Loaded  et_edt-ud-dev_2_am.json
 Loaded  et_edt-ud-dev_3_am.json
 Loaded  et_edt-ud-dev_4_am.json
 Loaded  et_edt-ud-test_21_am.json
 Loaded  et_edt-ud-test_30_am.json
 Loaded  et_edt-ud-test_5_am.json
 Loaded  et_edt-ud-test_6_am.json
 Loaded  et_edt-ud-test_7_am.json
 Loaded  et_edt-ud-train_10_am.json
 Loaded  et_edt-ud-train_11_am.json
 Loaded  et_edt-ud-train_12_am.json
 Loaded  et_edt-ud-train_13_am.json
 Loaded  et_edt-ud-train_14_am.json
 Loaded  et_edt-ud-train_15_am.json
 Loaded  et_edt-ud-train_16_am.json
 Loaded  et_edt-ud-train_17_am.json
 Loaded  et_edt-ud-train_18_am.json
 Loaded  et_edt-ud-train_22_am.json
 Loaded  et_edt-ud-train_23_am.json
 Loaded  et_edt-ud-train_24_am.json
 Loaded  et_edt-ud-train_25_am.json
 Loaded  et_edt-ud-train_26_am.json
 Loaded  et_edt-ud-train_27_am.json
 Loaded  et_edt-ud-tr

In [8]:
# Corpus with gold standard annotations and automaatmärgendus
input_dir = 'Automaatmargendusega_EWT_hastaalfinaal2'
assert os.path.isdir( input_dir )
loaded_texts_ewt_am = []
for fname in os.listdir( input_dir ):
    #if 'dev' in fname:
        #continue
    #if 'test' in fname:
     #   continue
    #if 'train' in fname:
     #   continue
    if fname.endswith('.json'):
        # Load Text with gold standard annotations
        text = json_to_text(file=os.path.join(input_dir, fname) )
        loaded_texts_ewt_am.append( text )
        print(' Loaded ', fname)

 Loaded  et_ewt-ud-dev_0_am.json
 Loaded  et_ewt-ud-dev_1_am.json
 Loaded  et_ewt-ud-dev_7_am.json
 Loaded  et_ewt-ud-dev_8_am.json
 Loaded  et_ewt-ud-dev_9_am.json
 Loaded  et_ewt-ud-test_10_am.json
 Loaded  et_ewt-ud-test_11_am.json
 Loaded  et_ewt-ud-test_12_am.json
 Loaded  et_ewt-ud-test_13_am.json
 Loaded  et_ewt-ud-test_14_am.json
 Loaded  et_ewt-ud-test_15_am.json
 Loaded  et_ewt-ud-test_2_am.json
 Loaded  et_ewt-ud-train_16_am.json
 Loaded  et_ewt-ud-train_17_am.json
 Loaded  et_ewt-ud-train_18_am.json
 Loaded  et_ewt-ud-train_19_am.json
 Loaded  et_ewt-ud-train_20_am.json
 Loaded  et_ewt-ud-train_21_am.json
 Loaded  et_ewt-ud-train_22_am.json
 Loaded  et_ewt-ud-train_23_am.json
 Loaded  et_ewt-ud-train_24_am.json
 Loaded  et_ewt-ud-train_25_am.json
 Loaded  et_ewt-ud-train_26_am.json
 Loaded  et_ewt-ud-train_27_am.json
 Loaded  et_ewt-ud-train_28_am.json
 Loaded  et_ewt-ud-train_29_am.json
 Loaded  et_ewt-ud-train_30_am.json
 Loaded  et_ewt-ud-train_31_am.json
 Loaded  et_ewt

In [9]:
#jaotab korpuse žanripõhiselt eraldi listidesse
aja = [] #ajakirjandus
tea = [] #teadus
ilu = [] #ilukirjandus
for t in loaded_texts_edt_am:
    file_id = t.meta['file_prefix']
    if re.match('^tea', file_id):
        tea.append(t)
    elif re.match('^aja', file_id):
        aja.append(t)
    elif re.match('^ilu', file_id):
        ilu.append(t)
        
foorum = [] 
ewtb = [] #estonian web treebank (vanad tekstid)
kom = [] #kommentaarium
for t in loaded_texts_ewt_am:
    file_id = t.meta['file_prefix']
    if 'foorum' in file_id:
        foorum.append(t)
    elif 'ewtb' in file_id:
        ewtb.append(t)
    elif 'kom' in file_id:
        kom.append(t)

In [10]:
#KORPUSEPÕHISED

eel_jarel_liit.tag(aja)
eel_jarel_liit.tag(tea)
eel_jarel_liit.tag(ilu)
eel_jarel_liit.tag(foorum)
eel_jarel_liit.tag(ewtb)
eel_jarel_liit.tag(kom)

edt_korpusepohistega = aja + ilu + tea
ewt_korpusepohistega = foorum + ewtb + kom

In [17]:
output_dir = 'Automaatmargendusega_EDT_hastaalfinaal3'
if not os.path.isdir( output_dir ):
    os.makedirs(output_dir)
assert os.path.isdir( output_dir )

In [18]:
# Load gold standard texts and add pre-annotations
i = -1
for tekst in edt_korpusepohistega:
    i += 1
    if fname.endswith('.json'):
        fname = tekst.meta['file']
        fname2 = '_' + str(i) + '_am.json' 
        fname = fname.replace('.conllu', fname2)
        fpath = os.path.join(output_dir, fname)
        text_to_json(tekst, file=fpath)
        print(' Loaded, pre-annotated and written', fname)

 Loaded, pre-annotated and written et_edt-ud-dev_0_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_1_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_2_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_3_am.json
 Loaded, pre-annotated and written et_edt-ud-dev_4_am.json
 Loaded, pre-annotated and written et_edt-ud-test_5_am.json
 Loaded, pre-annotated and written et_edt-ud-test_6_am.json
 Loaded, pre-annotated and written et_edt-ud-test_7_am.json
 Loaded, pre-annotated and written et_edt-ud-train_8_am.json
 Loaded, pre-annotated and written et_edt-ud-train_9_am.json
 Loaded, pre-annotated and written et_edt-ud-train_10_am.json
 Loaded, pre-annotated and written et_edt-ud-train_11_am.json
 Loaded, pre-annotated and written et_edt-ud-train_12_am.json
 Loaded, pre-annotated and written et_edt-ud-train_13_am.json
 Loaded, pre-annotated and written et_edt-ud-train_14_am.json
 Loaded, pre-annotated and written et_edt-ud-train_15_am.json
 Loaded, pre-annotated and writ

In [19]:
output_dir = 'Automaatmargendusega_EWT_hastaalfinaal3'
if not os.path.isdir( output_dir ):
    os.makedirs(output_dir)
assert os.path.isdir( output_dir )

In [20]:
# Load gold standard texts and add pre-annotations
i = -1
for tekst in ewt_korpusepohistega:
    i += 1
    if fname.endswith('.json'):
        fname = tekst.meta['file']
        fname2 = '_' + str(i) + '_am.json' 
        fname = fname.replace('.conllu', fname2)
        fpath = os.path.join(output_dir, fname)
        text_to_json(tekst, file=fpath)
        print(' Loaded, pre-annotated and written', fname)

 Loaded, pre-annotated and written et_ewt-ud-dev_0_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_1_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_2_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_3_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_4_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_5_am.json
 Loaded, pre-annotated and written et_ewt-ud-train_6_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_7_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_8_am.json
 Loaded, pre-annotated and written et_ewt-ud-dev_9_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_10_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_11_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_12_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_13_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_14_am.json
 Loaded, pre-annotated and written et_ewt-ud-test_15_am.json
 Loaded, pre-annotated and written 